<a href="https://colab.research.google.com/github/hillmydyan/kominfo-/blob/main/tubes_NLP_awkoakowkaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Library

In [ ]:
!pip install transformers datasets evaluate sentencepiece sacrebleu


In [ ]:
!pip install rouge_score sacrebleu evaluate


In [ ]:
!pip install -q rouge_score sacrebleu evaluate


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00


Import library

In [ ]:
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)
import evaluate
from google.colab import files
from sklearn.model_selection import train_test_split


upload dataset

In [ ]:
uploaded = files.upload()
dataset_path = list(uploaded.keys())[0]
dataset_path


Saving indonesian_test.jsonl to indonesian_test.jsonl
Saving indonesian_train.jsonl to indonesian_train.jsonl
Saving indonesian_val.jsonl to indonesian_val.jsonl


'indonesian_test.jsonl'

load dataset

In [ ]:
def load_dataset(path):
    df = pd.read_json(path, lines=True)
    df = df.rename(columns={"text": "text", "summary": "summary"})
    df = df.dropna(subset=["text", "summary"])
    return Dataset.from_pandas(df)

raw_dataset = load_dataset(dataset_path)
raw_dataset


Dataset({
    features: ['id', 'url', 'title', 'summary', 'text'],
    num_rows: 1000
})

pre processing

In [ ]:
def clean_text(text):
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    return text.strip()

def preprocess_dataset(dataset):
    return dataset.map(lambda x: {
        "text": clean_text(x["text"]),
        "summary": clean_text(x["summary"])
    })

clean_dataset = preprocess_dataset(raw_dataset)
clean_dataset


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'url', 'title', 'summary', 'text'],
    num_rows: 1000
})

Train/Test Split

In [ ]:
df = clean_dataset.to_pandas()

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
})
dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', '__index_level_0__'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', '__index_level_0__'],
        num_rows: 200
    })
})

### Hasil Evaluasi Model

**ROUGE Score:**
*   **ROUGE-1:** 0.0002777777777777778
*   **ROUGE-2:** 0.0
*   **ROUGE-L:** 0.0002777777777777778
*   **ROUGE-Lsum:** 0.0002777777777777778

**BLEU Score:**
*   **BLEU:** 0.0
*   **Precisions:** [0.13333333333333333, 0.0, 0.0, 0.0]
*   **Brevity Penalty:** 2.869557223043413e-137
*   **Length Ratio:** 0.0031705770450221942
*   **Translation Length:** 15
*   **Reference Length:** 4731

### Kesimpulan dan Analisis

Berdasarkan metrik ROUGE dan BLEU, model T5 yang telah dilatih menunjukkan performa yang sangat rendah dalam menghasilkan ringkasan:

*   **ROUGE Score** yang mendekati nol (terutama ROUGE-2 dan ROUGE-L) menunjukkan bahwa ada sangat sedikit tumpang tindih kata atau frasa antara ringkasan yang dihasilkan model dengan ringkasan referensi. Ini mengindikasikan bahwa model tidak mampu menangkap konten inti atau struktur kalimat dari ringkasan target.

*   **BLEU Score** sebesar 0.0 juga menegaskan bahwa kualitas ringkasan yang dihasilkan sangat buruk. Skor ini mengukur seberapa dekat terjemahan mesin (dalam kasus ini, ringkasan yang dihasilkan) dengan terjemahan manusia (ringkasan referensi) berdasarkan presisi n-gram. Nilai nol berarti tidak ada tumpang tindih n-gram yang signifikan.

### Faktor Penyebab Potensial:
1.  **Jumlah Epoch yang Sedikit:** Model hanya dilatih selama 1 epoch (`num_train_epochs=1`). Untuk tugas summarization yang kompleks, ini biasanya tidak cukup untuk model belajar pola yang efektif dari data.
2.  **Ukuran Dataset Kecil:** Jika dataset pelatihan (`tokenized_dataset["train"]` dengan 800 sampel) relatif kecil untuk kompleksitas model T5, model mungkin kesulitan untuk melakukan generalisasi.
3.  **Hyperparameter Tuning:** Parameter pelatihan seperti `per_device_train_batch_size` dan `max_length` mungkin perlu disesuaikan. Ukuran batch 2 sangat kecil dan mungkin tidak optimal untuk pelatihan.
4.  **Tokenisasi dan Preprocessing:** Meskipun ada preprocessing, bisa jadi ada kebutuhan penyesuaian lebih lanjut pada langkah tokenisasi atau pembersihan data.

### Rekomendasi Selanjutnya:
1.  **Tingkatkan Jumlah Epoch:** Latih model dengan jumlah epoch yang lebih banyak (misalnya, 3-10 epoch) dan pantau kurva loss untuk melihat apakah model mulai belajar.
2.  **Sesuaikan Hyperparameter:** Eksperimen dengan ukuran batch yang lebih besar jika memori GPU memungkinkan, atau sesuaikan learning rate.
3.  **Eksplorasi Model yang Lebih Kecil atau Lebih Besar:** Pertimbangkan menggunakan model T5 yang lebih kecil atau lebih besar tergantung pada ketersediaan sumber daya dan kompleksitas tugas.
4.  **Analisis Error:** Periksa beberapa contoh ringkasan yang dihasilkan model secara manual untuk memahami jenis kesalahan yang dibuatnya.

Tokenisasi

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def tokenize_batch(batch):
    model_inputs = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    labels = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_batch, batched=True)
tokenized_dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

load model dan training setup

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="t5_output",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    logging_steps=50,
    report_to="none"
)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

training model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/tmp/ipython-input-3084402752.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,1.873200,1.763657


TrainOutput(global_step=400, training_loss=2.233977642059326, metrics={'train_runtime': 1803.3183, 'train_samples_per_second': 0.444, 'train_steps_per_second': 0.222, 'total_flos': 108273441177600.0, 'train_loss': 2.233977642059326, 'epoch': 1.0})

Evauasi ROGUE + BLEU

In [ ]:
def evaluate_model(model, dataset, tokenizer):
    preds = []
    refs = []

    model.eval()
    model.to("cpu")  # Change to CPU for AMD systems

    for example in dataset:
        input_text = example["text"]       # FIX SESUAI DATASET
        ref_text = example["summary"]      # FIX SESUAI DATASET

        tokenized = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        )

        # Pindahkan input ke CPU
        input_ids = tokenized["input_ids"].to(model.device)
        attention_mask = tokenized["attention_mask"].to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128
            )

        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        preds.append(pred)
        refs.append(ref_text)

    rouge_result = evaluate.load("rouge").compute(predictions=preds, references=refs) # Load rouge here
    bleu_result = evaluate.load("bleu").compute(predictions=preds, references=[[r] for r in refs]) # Load bleu here

    return rouge_result, bleu_result

In [ ]:
rouge_score, bleu_score = evaluate_model(model, dataset["test"], tokenizer)
print("ROUGE:", rouge_score)
print("BLEU:", bleu_score)

ROUGE: {'rouge1': np.float64(0.0002777777777777778), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0002777777777777778), 'rougeLsum': np.float64(0.0002777777777777778)}
BLEU: {'bleu': 0.0, 'precisions': [0.13333333333333333, 0.0, 0.0, 0.0], 'brevity_penalty': 2.869557223043413e-137, 'length_ratio': 0.0031705770450221942, 'translation_length': 15, 'reference_length': 4731}
